$UCB = Q(s,a) + \left(c \cdot P(s,a) \cdot \sqrt{\frac{lnN} {1+n}}\right)$

$Q = \text{value}$ <br>
$P = \text{policy}$ <br>
$N = \text{parent node visits}$ <br>
$n = \text{action visits}$ <br>
$c = \text{exploration hyperparameter}$ <br>
$a = \text{current action}$ <br>
$s = \text{current state}$

<br>
<br>
<br>


$pUCT = Q(s,a) + P(s,a) \cdot \frac{\sqrt{\sum_{b}{N(s,b)}}} {1+N(s,a)} \left(c_{1} + \text{log} \left(\frac{\sum_{b}{N(s,b)+c_{2}+1}} {c_{2}}\right) \right)$

$Q = \text{value}$ <br>
$P = \text{policy}$ <br>
$N = \text{visit count}$ <br>
$c_{1} = \text{exploration hyperparameter}$ <br>
$c_{2} = \text{exploration hyperparameter}$ <br>
$a = \text{current action}$ <br>
$b = \text{avaliable actions}$ <br>
$s = \text{current state}$


<br>
<br>
<br>

$P(s,a) = (1-\epsilon)p_{a} + \epsilon \eta_{a}$

$\epsilon = \text{exploration factor}$ <br>
$\eta = \text{dirchlet distribution}$


<br>
<br>
<br>

$\eta \sim \text{Dir}(\alpha)$

$\alpha = \text{vector of repetitions}$
<br>
<br>
<br>


$\overline{\rm Q}(s^{k-1},a^{k}) = \frac{Q(s^{k-1},a^{k})-\text{min}_{s,a \epsilon Tree}Q(s,a)}{\text{max}_{s,a \epsilon Tree}Q(s,a)-\text{min}_{s,a \epsilon Tree}Q(s,a)}$

$Q = \text{value}$ <br>
$s = \text{state}$ <br>
$a = \text{action}$


<br>
<br>
<br>


$G^{k} = \sum^{l-1-k}_{\tau=0}{\gamma^{\tau}r_{k+1+\tau}+\gamma^{l-k}v^{l}}$

$\gamma = \text{gamma discount}$ <br>
$r = \text{reward}$ <br>
$v = \text{value}$

<br>
<br>
<br>

$Q(s^{k-1},a^{k}):=\frac{N(s^{k-1},a^{k}) \cdot Q(s^{k-1},a^{k})+G^{k}}{N(s^{k-1},a^{k})+1}$

$Q = \text{value}$ <br>
$N = \text{visit count}$ <br>
$G = \text{generalized backup}$ <br>
$s = \text{state}$ <br>
$a = \text{action}$

<br>
<br>
<br>

$h_{\theta}(s) = s_{h}$

$s = \text{current state}$ <br>
$s_{h} = \text{hidden state representation}$

<br>
<br>
<br>

$f_{\theta}(s^{k}) = p^{k},v^{k}$

$p = \text{policy}$ <br>
$v = \text{value}$ <br>



<br>
<br>
<br>

$g_{\theta}(s_{h}^{k-1},a^{k}) = r^{k},s_{h}^{k}$

$a = \text{current action}$ <br>
$s_{h} = \text{hidden state representation}$ <br>
$r = \text{reward}$

In [1]:
import numpy as np
import math
import random
import pandas as pd

class MCTS:
    """
    Monte Carlo Tree Search algorithm used to search game tree for the best move
    """
    def __init__(
        self, 
        #prediction,
        #dynamics,
        user = None,
        c1 = 1.25,
        c2 = 19652,
        d_a = .3,
        e_f = .25,
        g_d = 1.,
        temp = 1.,
        max_depth = float('inf')
    ):
        """
        Input: prediction - NN used to predict p, v values
               dynamics - NN used to predict the next states hidden values and the reward
               user - integer representing which user the agent is (default = None) [OPTIONAL]
               c1 - float representing a search hyperparameter (default = 1.25) [OPTIONAL]
               c2 - float representing a search hyperparameter (default = 19652) [OPTIONAL]
               d_a = float representing the dirichlet alpha you wish to use (default = 0.3) [OPTIONAL]
               e_f = float representing the exploration fraction you wish to use with dirichlet alpha noise (default = 0.25) [OPTIONAL]
               g_d = float representing the gamma discount to apply to v_k+1 when backpropagating tree (default = 1) [OPTIONAL]
               max_depth - integer representing the max allowable depth to search tree (default = inf) [OPTIONAL]
        Description: MCTS initail variables
        Output: None
        """
        self.tree = {} #Game tree
        self.l = 0 #Last node depth
        self.max_depth = max_depth #Max allowable depth
        self.c1 = c1 #Exploration hyper parameter 1
        self.c2 = c2 #Exploration hyper parameter 2
        self.d_a = d_a #Dirichlet alpha
        self.e_f = e_f #Exploration fraction
        self.g_d = g_d #Gamma discount
        self.Q_max = 1 #Max value
        self.Q_min = -1 #Min value
        self.v_l = 0 #Last depth value
        #self.g = dynamics #Model used for dynamics
        #self.f = prediction #Model used for prediction

    class Node:
        """
        Node for each state in the game tree
        """
        def __init__(self):
            """
            Input: None
            Description: Node initail variables
            Output: None
            """
            self.N = 0 #Visits
            self.Q = 0 #Value
            self.R = 0 #Reward
            self.S = None #State
            self.P = None #Policy
            self.n_s = []
            
    def state_hash(self, s):
        """
        Input: s - tensor representing hidden state of task
        Description: generate unique hash of the supplied hidden state
        Output: integer representing unique hash of the supplied hidden state
        """
        result = hash(str(s))
        return result
    
    def dirichlet_noise(self, p):
        """
        Input: p - list of floats [0-1] representing action probability distrabution
        Description: add dirichlet noise to probability distrabution
        Output: list of floats [0-1] representing action probability with added noise
        """
        d = np.random.dirichlet([self.d_a] * len(p))
        return (d * self.e_f) + ((1 - self.e_f) * p)
    
    def pUCT(self, s):
        """
        Input: s - tensor representing hidden state of task
        Description: return best action state using polynomial upper confidence trees
        Output: integer containing the best action
        """
        p_visits = sum([self.tree[(s, b)].N for b in range(4)]) #Sum of all potential nodes
        #p_visits = sum([self.tree[(s, b)].N for b in range(self.f.action_space)]) #Sum of all potential nodes
        u_bank = {}
        #for a in range(self.f.action_space):
        for a in range(4):
            U = self.tree[(s, a)].P * ((p_visits**(0.5))/(1+self.tree[(s, a)].N)) #First part of exploration
            U *= self.c1 + (math.log((p_visits+(4*self.c2)+4)/self.c2)) #Second part of exploration
            #U *= self.c1 + (math.log((p_visits + (self.f.action_space * self.c2) + self.f.action_space) / self.c2)) #Second part of exploration
            Q_n = (self.tree[(s, a)].Q - self.Q_min) / (self.Q_max - self.Q_min) #Normalized value
            u_bank[a] = Q_n + U
        a_bank = [k for k,v in u_bank.items() if v == max(u_bank.values())]
        return random.choice(a_bank)
        
    def search(self, s, a = None, train = False):
        """
        Input: s - tensor representing hidden state of task
               a - integer representing which action is being performed (default = None) [OPTIONAL]
               train - boolean representing if search is being used in training mode (default = False) [OPTIONAL]
        Description: Search the task action tree using upper confidence value
        Output: predicted value
        """
        s_hash = self.state_hash(s) #Create hash of state [sk-1] for game tree
        if (s_hash, a) not in self.tree:
            self.tree[(s_hash, a)] = self.Node() #Initialize new game tree node
        if a is not None:
            if self.tree[(s_hash, a)].S is None:
                r_k, s = self.g(s, a) #Reward and state prediction using dynamics function
                self.tree[(s_hash, a)].S = s
                self.tree[(s_hash, a)].R = r_k
            else:
                s = self.tree[(s_hash, a)].S
        sk_hash = self.state_hash(s) #Create hash of state [sk] for game tree
        if self.tree[(s_hash, a)].N == 0:
            #EXPANSION ---
            v_k, p = self.f(s) #Value and policy prediction using prediction function
            if a is None and train == True:
                p = self.dirichlet_noise(p) #Add dirichlet noise to p @ s0
            self.tree[(s_hash, a)].Q = v_k
            self.v_l = self.tree[(s_hash, a)].Q
            for a_k, p_a in enumerate(p):
                self.tree[(sk_hash, a_k)] = self.Node()
                self.tree[(sk_hash, a_k)].P = p_a
                self.tree[(s_hash, a)].n_s.append((sk_hash, a_k))
        elif self.l < self.max_depth:
            a_k = self.pUCT(sk_hash) #Find best action to perform @ [sk]
            k = self.l
            self.l += 1
            #BACKUP ---
            G_1, r_1 = self.search(s, a_k) #Go level deeper
            if (self.l - k - 1) > 0:
                G_k = ((self.g_d ** (self.l - k - 1)) * r_1) + ((self.g_d ** (self.l - k)) * self.v_l)
                G_k += G_1
            else:
                G_k = 0
            self.tree[(s_hash, a)].Q = ((self.tree[(s_hash, a)].N * self.tree[(s_hash, a)].Q) + G_k) / (self.tree[(s_hash, a)].N + 1) #Updated value
        if self.tree[(s_hash, a)].Q < self.Q_min:
            self.Q_min = self.tree[(s_hash, a)].Q
        if self.tree[(s_hash, a)].Q > self.Q_max:
            self.Q_max = self.tree[(s_hash, a)].Q
        if 'G_k' not in locals():
            G_k = 0
        self.tree[(s_hash, a)].N += 1
        return G_k, -self.tree[(s_hash, a)].R
    
    def g(self,s,a):
        """
        TEMPORARY DYNAMICS MODEL FOR TESTING SEARCH
        """
        return 0, s*random.choice([.1, .2, .11, .26, .4, .7, .6, .3, .5, .66, .8, .14, .32, .17])
    
    def f(self, s):
        """
        TEMPORARY PREDICTION MODEL FOR TESTING SEARCH
        """
        return random.choice([-1, 0, 1]), np.random.rand(4)
    
    def log(self):
        pass

$MCTS(s_{t},\mu_{\theta})=v_{t},\pi_{t}$

$s = \text{current state}$ <br>
$v = \text{state value}$ <br>
$\pi = \text{action policy}$ <br>
$\mu_{\theta} = \text{neural networks}$

<br>
<br>

$\mu_{\theta} = h_{\theta}(s),f_{\theta}(s^{k}),g_{\theta}(s_{h}^{k-1},a^{k})$

$\mu_{\theta} = \text{neural networks}$ <br>
$h_{\theta} = \text{representation}$ <br>
$f_{\theta} = \text{predictions}$ <br>
$g_{\theta} = \text{dynamics}$

<br>
<br>

$\pi=\frac{N(a)^{1/\tau}}{\sum_{b}{N(b)^{1/\tau}}}$

$N = \text{visit count}$ <br>
$\tau = \text{tempature}$ <br>
$a = \text{current action}$ <br>
$b = \text{avaliable actions}$

In [2]:
temp = 1
num_sims = 25
s = np.random.rand(3,2)

search = MCTS()
for x in range(num_sims):
    search.l = 0
    search.v_l = 0
    search.search(s)

s_hash = search.state_hash(s)
counts = {a: search.tree[(s_hash,a)].N for a in range(4)}

if temp == 0:
    a_bank = [k for k,v in counts.items() if v == max(counts.values())]
    a = random.choice(a_bank)
    probs = [0] * len(counts)
    probs[a] = 1
else:
    c_s = sum(c ** (1./temp) for c in counts.values())
    probs = [(x ** (1./temp)) / c_s for x in counts.values()]
    
print(probs)

[0.20833333333333334, 0.2916666666666667, 0.375, 0.125]


In [6]:
import matplotlib.pyplot as plt
import networkx as nx
import pydot
from networkx.drawing.nx_pydot import graphviz_layout

G = nx.Graph()
for s in search.tree:
    for a in search.tree[s].n_s:
        G.add_edge(s,a)
        
pos = graphviz_layout(G, prog='dot')
#Display network graph -----------------------------
nx.draw(
    G, #Graph nodes & connections
    pos, #Position of graph
    with_labels=True #Labels on nodes
)
plt.rcParams['figure.figsize'] = [40, 40] #Resize graph
plt.show()

FileNotFoundError: [WinError 2] "dot" not found in path.